# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

  - 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

> NOTE: If you're running this locally - please skip this step.

In [4]:
%pip install -qU langchain langchain_openai langchain_community langgraph arxiv langchain-google-community

/Users/kcheng_homelaptop/Workspace/ai-maker_space/AIE5/05_Our_First_Agent_with_LangGraph/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [4]:
os.environ["GOOGLE_CSE_ID"] = getpass.getpass("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = getpass.getpass("GOOGLE_API_KEY")

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE5 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [8]:
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

google_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [43]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
    google_tool
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [44]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [45]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

**Answer**:
1. Tool Descriptions: Each tool in the action node includes:
- Name: A concise name for the tool(e.g. "google_search")
- Description: A detailed explanation of what the tool does (e.g., "Search Google for recent results.), its inputs and outputs
- Parameters: If the tool requires any input parameters (e.g., string of query)
2. Model Prompting: When the model node is executed, we need to provide it with a prompt that:
- Clearly states the overal task. What needs to be accomplished?
- Lists the available tools. Inlude the name and description of each tool in the action node.
- Instructs the model to choose the best tool. Tells the model to analyze the task and select the tool that is most likely to help achieve the goal.
3. Model Ouput: the model's output should indicate which tool it has chosen with related parameters. 
4. Action Node Execution: the action node receives the model's output. It parses this output to
- identify the selected tool
- Extract any arguments that need to be passed to the tool
- Execute the chosen tool with the provided arguments

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [47]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [48]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [49]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [50]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [51]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [52]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [53]:
compiled_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

**Answer**:
Uncontrolled cycles can lead to infinite loops and the programg hanging. LangGraph itself does not inherently impose a limit on cycle time.

If not, how could we impose a limit to the number of cycles?

**Answer**:
- Counter within a Node:  The most common and generally preferred method is to implement a counter within the node that's part of the cycle. It increments the counter on each iteration. When the counter reaches the limit, the node returns a done signal, which is used as a condition to exit the cycle.
- Conditional Edges: Langflow's conditional edges are crucial for controlling cycles. We can use conditions to determine whether an edge should be followed.  This is how we may break the cycle based on the counter or some other criteria.  Make sure we have a clear exit condition. 


## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:
1. First question, "Who is the current captain of the Winnipeg Jets?"
2. Second question, "What is the paper DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning about?"

In [54]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is the paper DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning about?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4xV7LFICsyd95ZgunkckFC1B', 'function': {'arguments': '{"query":"DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 204, 'total_tokens': 237, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-342ffa07-cb02-4a21-9555-cd75e57d85f2-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning'}, 'id': 'call_4xV7LFICsyd95ZgunkckFC1B', 'type': 'tool_call'}], usage_

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [21]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Google!")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dtsHm4qm3hLY5Je3BD6h3awR', 'function': {'arguments': '{"query":"QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 204, 'total_tokens': 221, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-91bc13bd-7cc6-4e40-a4c0-40852e9d4884-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'QLoRA'}, 'id': 'call_dtsHm4qm3hLY5Je3BD6h3awR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 204, 'output_tokens': 17, 'total_tokens': 221, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'a

####🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

**Answer**:
1. Initial Input:

The HumanMessage containing the user's query was the initial input to the LangGraph.  It arrived at the first node, which we've designated as the "agent" node.

2. Agent Node Received Input:

The agent node received the HumanMessage.

3. Agent Node Constructed Prompt:

The agent node did not just send the raw HumanMessage to the language model. It constructed a carefully engineered prompt. This prompt included:

User's Instruction: The original HumanMessage was included in the prompt.
Context: empty string because there was no prior context.
Instructions to the LLM: The agent provided instructions to the LLM, such as:
"Decide which tool(s) to use, if any, to answer the user's question. If you need to use a tool, respond with a JSON object in the following format: {"tool": "tool_name", "arguments": {"arg1": "value1", ...}}."

4. Agent Node Sended Prompt to LLM:

The agent node sended thed constructed prompt to the language model (gtp-4o).

5. LLM Processed Prompt:

The LLM received the prompt, including the user's request and the tool descriptions. It analyzed the request and determined the best course of action.

6. LLM Returned Tool Call:

The LLM recognized the need to search arXiv, it returned a JSON object indicating a call to the arxiv tool:

tool_calls=[{'name': 'arxiv', 'args': {'query': 'QLoRA'}, 'id': 'call_dtsHm4qm3hLY5Je3BD6h3awR', 'type': 'tool_call'}]

7. Agent Node Received LLM Output:

The agent node received the above JSON object.

8. Agent Node Followed Conditional Edge:

The agent node saw that the LLM has specified a tool call. Because of the conditional edge from agent node to action node, the flow went to action node.

9. Action Node Executed Tool:

The agent node extracted the tool name ("arxiv") and the arguments ({"query": "QLoRA paper"}). It then executed the arxiv tool with this query.

10. Action Node Returned Tool Result:

The arxiv tool searched arXiv and returned the results (information about the QLoRA paper, including author names). The results were sent back to the agent node via the edge from action node to agent node.

11. Agent Node Received Tool Result:

The agent node received the results from the arxiv tool.

12. Agent Node Constructed New Prompt (Iteration 2):

The agent node now had the author names from the QLoRA paper. It needed to search Google for their tweets.  It constructed a new prompt for the LLM. This prompt included:

Previous Context: The results from the arxiv search (the author names) were included as context.
Instructions: The instructions told the LLM to use the google_search tool to find the latest tweets for each author.
Tool Descriptions: The google_search tool was described.

13. Steps 4-12 Repeat:

Steps 4-12 repeat for each author. The LLM will likely return multiple tool calls (one for each author), and the agent node will execute the google_search tool multiple times.

14. Agent Node Receives All Tool Results:

The agent node receives the results from all the google_search calls (the tweets).

15. Agent Node Constructs Final Response:

The agent node now has all the information it needs. It constructs a final prompt for the LLM, which will include:

All Context: The original HumanMessage, the arxiv results, and the google_search results (the tweets) are all included as context.
Instructions: The instructions will tell the LLM to synthesize all this information into a coherent answer for the user.
16. LLM Generates Final Answer:

The LLM generates the final answer, summarizing the QLoRA paper and the authors' latest tweets.

17. Agent Node Returns Final Answer:

The agent node receives the final answer from the LLM and returns it as the output of the LangGraph execution.



## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [22]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | compiled_graph | parse_output

In [23]:
agent_chain.invoke({"question" : "What is RAG?"})

"RAG stands for Retrieval-Augmented Generation. It is a technique used in natural language processing (NLP) that combines retrieval-based methods with generative models to improve the quality and relevance of generated text. Here's a brief overview of how it works:\n\n1. **Retrieval**: In the first step, relevant documents or pieces of information are retrieved from a large corpus or database. This is typically done using a retrieval model that identifies the most relevant documents based on the input query or context.\n\n2. **Augmentation**: The retrieved documents are then used to augment the input to a generative model. This means that the generative model has access to additional context or information that can help it produce more accurate and contextually relevant responses.\n\n3. **Generation**: Finally, a generative model, such as a transformer-based language model, uses the augmented input to generate a response or piece of text. The presence of additional context from the ret

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

####🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [24]:
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored OLoRA paper?",
    "What is the most populer deep learning framework?",
    "What significant improvements does the LoRA system make?"
    ### YOUR CODE HERE
]

answers = [
    {"must_mention": ["paged", "optimizer"]},
    {"must_mention": ["NF4", "NormalFloat"]},
    {"must_mention": ["ground", "context"]},
    {"must_mention": ["Tim", "Dettmers"]},
    {"must_mention": ["PyTorch","TensorFlow"]},
    {"must_mention": ["reduce", "parameters"]},
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [25]:
from langsmith import Client

client = Client()

dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [26]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [27]:
experiment_results = client.evaluate(
    agent_chain,
    data=dataset_name,
    evaluators=[must_mention],
    experiment_prefix=f"RAG Pipeline - Evaluation - {uuid4().hex[0:4]}",
    metadata={"version": "1.0.0"},
)

View the evaluation results for experiment: 'RAG Pipeline - Evaluation - 553c-a6b1ff79' at:
https://smith.langchain.com/o/48c839b0-a50d-4327-8b62-d77e548f014c/datasets/92c8b3bf-ec19-4f0f-ad05-37eaf3a60760/compare?selectedSessions=b8ea10e5-4107-4254-8812-db5b5b4c042c




0it [00:00, ?it/s]

In [28]:
experiment_results

<ExperimentResults RAG Pipeline - Evaluation - 553c-a6b1ff79>

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [29]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

####🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE

In [30]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### YOUR MARKDOWN HERE

In [31]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE

In [33]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

####🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

##### YOUR MARKDOWN HERE

In [34]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE

In [35]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE

In [36]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE

In [37]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hY9RArX3UOaxEvCcgSo0Hbrk', 'function': {'arguments': '{"query": "LoRA in machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_VHK1iJv3gGk55yFAvZZVxVbJ', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_XH4stFFNHuZmHQCp7EgWpsPz', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 204, 'total_tokens': 285, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_rea

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [38]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [39]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

**Prompt Engineering**: Prompt engineering is the process of creating effective prompts that enable AI models to generate responses based on given inputs. It involves crafting questions or statements that guide AI-driven interactions, and is essential for tasks such as natural language processing (NLP), translation, and chatbots. [Learn more](https://www.geeksforgeeks.org/what-is-prompt-engineering-the-ai-revolution/).

**When Did It Become Popular?**: Prompt engineering has become an essential skill in the age of artificial intelligence. While its recent prominence is linked to the AI revolution, the concept of designing effective prompts has deep roots in human communication and development. [Explore more](https://promptly.substack.com/p/a-journey-through-the-history-of).



RAG, or Retrieval-Augmented Generation, is a method in natural language processing that combines retrieval-based and generation-based approaches to improve the quality and relevance of generated text. It involves